In [1]:
import numpy as np
import pickle
import cv2
import glob

from scipy.ndimage.measurements import label
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

%matplotlib inline

from vehicle_detection_utilities import *



# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='YUV', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    
    on_windows = []    
    for window in windows:
        
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))    
        features = get_img_features(test_img, **search_win_params_classify)        
        test_features = scaler.transform(features.reshape(1, -1))
        
        prediction = clf.predict(test_features)
        
        if prediction == 1:
            on_windows.append(window)
            
    return on_windows

#GENERATES WINDOWS
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    
    imcopy = np.copy(img)
    
    for bbox in bboxes:
        
        r=random.randint(0,255)
        g=random.randint(0,255)
        b=random.randint(0,255)        
        color=(r, g, b) 
        
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

# Classifier Code

In [2]:
import os, pickle, glob
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import importlib
from sklearn.utils import shuffle
from skimage.feature import hog
from scipy.ndimage.measurements import label
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

%matplotlib inline

########## GET THE DATA!! ################
non_car_data_folder = "./data/non-vehicles/Extras/*.png"
car_data_folder     = "./data/vehicles/vehicles/*.png"
car_names     = glob.glob(car_data_folder)
non_car_names = glob.glob(non_car_data_folder)
cars = []
notcars = []
for image in car_names:
    cars.append(image)
for image in non_car_names:
    notcars.append(image)

cars = shuffle(cars)
notcars = shuffle(notcars)

# Equal number of train and test images
minlen = min(len(cars),len(notcars))
cars    = cars[:minlen]
notcars = notcars[:minlen]
carts   = shuffle(cars)
notcars = shuffle(notcars)
##########################################
    
search_win_params_classify = {
'color_space' : 'YUV',
'spatial_size' : (16,16),
'orient' : 9,
'pix_per_cell' : 16,
'cell_per_block' : 2,
'hog_channel' : "ALL",
'spatial_feat':True, 
'hist_feat':True, 
'hog_feat':True
}

t=time.time()

path = "./carFeatures.p"
if os.path.isfile(path):
    with open(path, "rb") as f:
        try:
            processed_training_data = pickle.load(f)
        except StandardError: # so many things could go wrong, can't be more specific.
            pass 
with open(path, "wb") as f:
    car_features    = [cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB) for i in cars]
    notcar_features = [cv2.cvtColor(cv2.imread(i), cv2.COLOR_BGR2RGB) for i in notcars]
    car_features    = [get_img_features(i, **search_win_params_classify) for i in car_features]
    notcar_features = [get_img_features(i, **search_win_params_classify) for i in notcar_features]

    processed_training_data = [car_features,notcar_features]
    pickle.dump(processed_training_data, f)

t2 = time.time()
hogtime = round(t2-t, 2)
print(round(t2-t, 2), 'Seconds to extract HOG features...')

# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)
X_scaler = StandardScaler().fit(X)

scaled_X = X_scaler.transform(X)


y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
   scaled_X, y, test_size=0.2, random_state=rand_state)

svc = LinearSVC()

t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
accuracy = round(svc.score(X_test, y_test), 4)
accuracystr = "%.4f" % accuracy
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

/home/ron/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


68.96 Seconds to extract HOG features...
4.14 Seconds to train SVC...
Test Accuracy of SVC =  0.9804


In [8]:
from collections import deque
from scipy.ndimage.measurements import label

class DetectorMediator:
    def __init__(self):        
        
        
        self.heatmap = None
        self.image = None
        self.count = 0
        self.deque_len = 15
        self.boxes_que = deque(maxlen=self.deque_len)
    def add_heat(self, heatmap, bbox_list):
        
        for box in bbox_list:
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1            
        return heatmap
    
    def apply_threshold(self, threshold):
        
        checkmap = np.copy(self.heatmap)
        checkmap[checkmap <= threshold] = 0        
        return checkmap
    
    def generate_heatmap(self):
        '''
        Generates the heatmap
        
        Doesn't return anything because it updates an instance var.
        
        This may affect runtime
        '''
        self.heatmap = np.zeros_like(self.image[:,:,0]).astype(np.float64)
        for boxCoordList in self.boxes_que:
            self.heatmap = self.add_heat(self.heatmap, boxCoordList)
    
    def draw_labeled_bboxes(self, img, labels):
        # Iterate through all detected cars
        for car_number in range(1, labels[1]+1):
            # Find pixels with each car_number label value
            nonzero = (labels[0] == car_number).nonzero()
            # Identify x and y values of those pixels
            nonzeroy = np.array(nonzero[0])
            nonzerox = np.array(nonzero[1])
            # Define a bounding box based on min/max x and y
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image
            cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
        
        return img
        
    def detect_vehicles(self, input_image, box_list):
        
        self.count +=1
        
        self.image = input_image
        
        self.boxes_que.append(box_list)
        
        self.generate_heatmap()
        
        thresholded_img = self.apply_threshold( 7 )
        
        labels = label(thresholded_img)
        
        output_image = self.draw_labeled_bboxes(self.image, labels)
        
        return output_image

In [9]:
frameno = 0
frameno = (frameno + 1) % 5

def window_dim(output_img, frameno):
    
    w_dim = 64
    
    return slide_window(output_img, x_start_stop=[None, None], y_start_stop=[475, 700], 
                    xy_window=(w_dim, w_dim), xy_overlap=(0.65, .65)) 

In [10]:
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    
    draw_img = np.copy(img)
    #img = img.astype(np.float32)/255
    
    img_tosearch = img[ystart:ystop,:,:]
    
    ###### Convert Image to the space then break it up into channels
    ctrans_tosearch = convert_color(img_tosearch, color_space='YCrCb')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    ################################################################
    
    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    output_windows = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                output_windows.append(((xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart)) )
    return output_windows
    
ystart = 400
ystop = 656
scale = 1

#out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient=11, pix_per_cell=16, cell_per_block=2, spatial_size=(16,16), hist_bins=32)


In [11]:

draw_images = []

test_images = glob.glob('./test_pipeline/*.jpg')

detector = DetectorMediator()

t = search_win_params_classify

windows1 = slide_window(image, x_start_stop=[0, 1280], y_start_stop=[400,464], 
                    xy_window=(64,64), xy_overlap=(0.15, 0.15))
windows4 = slide_window(image, x_start_stop=[0, 1280], y_start_stop=[420,501], 
                    xy_window=(80,80), xy_overlap=(0.2, 0.2))
windows2 = slide_window(image, x_start_stop=[0, 1280], y_start_stop=[450,650], 
                    xy_window=(96,96), xy_overlap=(0.3, 0.3))
windows3 = slide_window(image, x_start_stop=[0, 1280], y_start_stop=[550,700], 
                    xy_window=(128,128), xy_overlap=(0.5, 0.5))
theWindows = windows1 + windows2 + windows3 + windows4


def pipeline(input_img):
    #test_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)     
    
    #currentWindows = window_dim(input_img, frameno)    
    box_list = search_windows(input_img, theWindows, svc, X_scaler,**search_win_params_classify)     
    
    #box_list = find_cars(input_img, ystart, ystop, scale, svc, X_scaler, t['orient'], t['pix_per_cell'], t['cell_per_block'], t['spatial_size'], hist_bins=32)
    
    out_img = detector.detect_vehicles(input_img, box_list)
    
    return out_img


Remember video reads in as RGB
cv2 reads in on RGB

In [ ]:

from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'results.mp4'
clip1 = VideoFileClip("project_video.mp4")
clip2 = clip1.subclip(35,45)
white_clip = clip2.fl_image(pipeline)
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video results.mp4
[MoviePy] Writing video results.mp4


 81%|████████  | 203/251 [01:33<00:22,  2.18it/s]